# 8.3 Objectives

In [1]:
# install dependencies
%pip install -q amplpy

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=['highs'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


The declaration of an objective function consists of one of the keywords `minimize`
or `maximize`, a name, a colon, and a linear expression in previously defined sets,
parameters, and variables. We have seen examples such as

```ampl
minimize Total_Cost: sum {j in FOOD} cost[j] * Buy[j];
```

and

```ampl
maximize Total_Profit:
  sum {p in PROD, t in 1..T}
    ( sum {a in AREA[p]} revenue[p,a,t] * Sell[p,a,t]
    - prodcost[p] * Make[p,t]
    - invcost[p] * Inv[p,t] );
```

The name of the objective plays no further role in the model, with the exception of
certain *columnwise* declarations to be introduced in Chapters 15 and 16 xTODO. Within
AMPL commands, the objective's name refers to its value. Thus, for example, after
solving a feasible instance of the diet model in
[Figure 2-1](../02/2_2_an_AMPL_model_for_the_diet_problem.ipynb#fig-2-1),
we could issue the command

```ampl
ampl.display('{j in FOOD} 100 * cost[j] * Buy[j] / Total_Cost')
```

which produces

```text
100*cost[j]*Buy[j]/Total_Cost [*] :=
BEEF 14.4845
CHK   4.38762
FISH  3.8794
HAM  24.4792
MCH  16.0089
MTL  16.8559
SPG  15.6862
TUR   4.21822
;
```

to show the percentage of the total cost spent on each food.

Although a particular linear program must have one objective function, a model may
contain more than one objective declaration. Moreover, any `minimize` or `maximize`
declaration may define an indexed collection of objective functions, by including
an indexing expression after the objective name. In these cases, you may issue an
`objective` command, before typing `solve`, to indicate which objective is to be
optimized.

As an example, recall that when trying to solve the model of
[Figure 2-1](../02/2_2_an_AMPL_model_for_the_diet_problem.ipynb#fig-2-1)
with the accompanying data, we found that no solution could satisfy all of the
constraints. We subsequently increased the sodium (`NA`) limit to 50000 to make a
feasible solution possible.

It is reasonable to ask: *How much of an increase in the sodium limit is really
necessary to permit a feasible solution?* For this purpose we can introduce a new
objective equal to the total sodium in the diet:

```ampl
minimize Total_NA: sum {j in FOOD} amt["NA", j] * Buy[j];
```

(We create this objective only for sodium, because we have no reason to minimize most
of the other nutrients.)

We can solve the linear program for total cost as before, since AMPL chooses the
model's first objective by default:

```ampl
ampl: model diet.mod;
ampl: data diet2a.dat;
ampl.display('n_max["NA"]')
n_max["NA"] = 50000

ampl: minimize Total_NA: sum {j in FOOD} amt["NA", j] * Buy[j];
ampl: solve;
```

The solver reports:

```text
MINOS 5.5: optimal solution found.
13 iterations, objective 118.0594032
Objective = Total_Cost
```

The solver tells us the minimum cost, and we can also use `display` to look at the
total sodium, even though it is not currently being minimized:

```ampl
ampl: display Total_NA;
```

```text
Total_NA = 50000
```

Next we use the `objective` command to switch the objective to minimization of total
sodium. The `solve` command then re-optimizes with this alternative objective, and we
display `Total_Cost` to determine the resulting cost:

```ampl
ampl: objective Total_NA;
ampl: solve;
```

```text
MINOS 5.5: optimal solution found.
1 iteration, objective 48186
```

```ampl
ampl: display Total_Cost;
```

```text
Total_Cost = 123.627
```

We see that sodium can be brought down by about 1 800 units, though the cost is forced
up by about $5.50 as a result. (Healthier diets are in general more expensive, because
they force the solution away from the one that minimizes cost.)

As another example, consider experimenting with different optimal solutions for the
office assignment problem. First we solve the original problem:

```ampl
ampl: model transp.mod;
ampl: data assign.dat;
ampl: solve;
```

```text
CPLEX 8.0.0: optimal solution; objective 28
24 dual simplex iterations (0 in phase I)
```

We then adjust display options and examine the solution:

```ampl
ampl: option display_1col 1000, omit_zero_rows 1;
ampl: option display_eps 0.000001;

ampl: display Total_Cost,
      {i in ORIG, j in DEST} cost[i,j] * Trans[i,j];
```

```text
Total_Cost = 28
cost[i,j]*Trans[i,j] :=
Coullard C118    6
Daskin    D241   4
Hazen     C246   1
Hopp      D237   1
Iravani   C138   2
Linetsky C250    3
Mehrotra D239    2
Nelson    C140   4
Smilowitz M233   1
Tamhane   C251   3
White     M239   1
;
```

To keep the objective value at this optimal level while we experiment, we add a
constraint that fixes the objective expression at its current value, 28:

```ampl
ampl: subject to Stay_Optimal:
       sum {i in ORIG, j in DEST} cost[i,j] * Trans[i,j] = 28;
```

Recall that `cost[i,j]` is the ranking that person `i` has given to office `j`, while
`Trans[i,j]` equals 1 if person `i` is assigned to office `j`, and 0 otherwise. Thus

```ampl
sum {j in DEST} cost[i,j] * Trans[i,j]
```

always equals the ranking of person `i` for the office to which that person is
assigned. We use this expression to declare a new indexed objective function:

```ampl
ampl: minimize Pref_of {i in ORIG}:
       sum {j in DEST} cost[i,j] * Trans[i,j];
```

This statement creates, for each person `i`, an objective `Pref_of[i]` that minimizes
the ranking of the office assigned to that person. We can then select one individual
and optimize their ranking:

```ampl
ampl: objective Pref_of["Coullard"];
ampl: solve;
```

```text
CPLEX 8.0.0: optimal solution; objective 3
3 simplex iterations (0 in phase I)
```

Examining the new assignment, we see that the original objective value is unchanged,
while the selected individual's assignment has improved, necessarily at the expense
of others:

```ampl
ampl: display Total_Cost,
      {i in ORIG, j in DEST} cost[i,j] * Trans[i,j];
```

```text
Total_Cost = 28
cost[i,j]*Trans[i,j] :=
Coullard D241    3
Daskin    D237   1
Hazen     C246   1
Hopp      C251   5
Iravani   C138   2
Linetsky C250    3
Mehrotra D239    2
Nelson    C140   4
Smilowitz M233   1
Tamhane   C118   5
White     M239   1
;
```

We were able to make this change because there are multiple optimal solutions to
the original total-ranking objective. A solver arbitrarily returns one of these, but by
introducing a second objective we can guide it toward others.
